In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import string
import nltk
import sklearn
import mglearn as mglearn

/home/swagmaster/anaconda3/envs/atab/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/home/swagmaster/anaconda3/envs/atab/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
train = pd.read_csv('train2electricboogaloo.csv')

train.Summary.fillna('', inplace=True)
train.Text.fillna('', inplace=True)
train.SumTxt.fillna('', inplace=True)

In [3]:
text, score = train['SumTxt'], train['Score']

In [4]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [5]:
stemmer = SnowballStemmer("english")

In [6]:
def text_process_stm(mess): #ss, swr
    nopunc = mess.translate(str.maketrans('', '', string.punctuation))
    stop_words = set(stopwords.words("english"))
    nopunc2 = [word for word in nopunc.split() if word.lower() not in stop_words]
    nopunc2 = [word.lower() for word in nopunc2]
    return [stemmer.stem(word) for word in nopunc2]

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
tfidfvect = TfidfVectorizer(analyzer = text_process_stm)

In [9]:
logregvect = tfidfvect.fit_transform(text)

In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [11]:
logreg = LogisticRegression(n_jobs = 6)

In [12]:
# from sklearn.base import is_classifier

In [13]:
# is_classifier(logreg)

In [14]:
# logregfit = logreg.fit(tfidfvectors, score)

In [15]:
# scores = cross_val_score(logreg, logregvect, score, cv=10)
# np.mean(scores)

In [16]:
scores = cross_val_score(logreg, logregvect, score, cv=5)
np.mean(scores)

0.750612187456021

In [17]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(train, test_size = 0.2, random_state = 42, stratify = train['Score'])
train_text, train_score = train_set['SumTxt'], train_set['Score']
test_text, test_score = test_set['SumTxt'], test_set['Score']

In [18]:
from sklearn.naive_bayes import MultinomialNB

In [23]:
mnbtrainvect = tfidfvect.fit_transform(train_text)
mnbtestvect = tfidfvect.transform(test_text)

In [24]:
mnb = MultinomialNB().fit(mnbtrainvect, train_score)

In [25]:
pred = mnb.predict(mnbtestvect)

In [26]:
from sklearn.metrics import classification_report

In [27]:
print (classification_report(test_score, pred))

              precision    recall  f1-score   support

           1       0.92      0.07      0.12      7838
           2       0.71      0.00      0.00      4471
           3       0.85      0.00      0.01      6399
           4       0.84      0.01      0.01     12062
           5       0.64      1.00      0.78     54498

    accuracy                           0.65     85268
   macro avg       0.79      0.21      0.18     85268
weighted avg       0.72      0.65      0.51     85268



In [28]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_jobs = 6)

In [29]:
scores = cross_val_score(neigh, logregvect, score, cv=10)
np.mean(scores)

0.7086597551250177

In [30]:
from sklearn.svm import LinearSVC

In [31]:
lsvm = LinearSVC()

In [32]:
scores = cross_val_score(lsvm, logregvect, score, cv=10)
np.mean(scores)

0.775528920579819

In [33]:
from sklearn.ensemble import RandomForestClassifier

In [34]:
rfc = RandomForestClassifier(n_jobs = 6)

In [35]:
scores = cross_val_score(rfc, logregvect, score, cv=10)
np.mean(scores)

0.7912511141342591